In [19]:
import numpy as np
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from auxFunc import entropydensity
from scipy.special import kn, zeta


In [20]:
from sympy.parsing.mathematica import parse_mathematica
from sympy import var
x = var('x')
expr = "-(F[x]*G[x]^2) - 2*F[x]*G[x]*Yeq[x] - (F[x]*Yeq[x]^2)/(1 + F[x]*Yeq[x])^2 + (2*F[x]*G[x]*Yeq[x])/(1 + F[x]*Yeq[x]) + (2*F[x]*Yeq[x]^2)/(1 + F[x]*Yeq[x]) -  (Yeq[x]^2*Derivative[1][F][x])/(1 + F[x]*Yeq[x])^2 - Derivative[1][Yeq][x] - (F[x]*Yeq[x]*Derivative[1][Yeq][x])/(1 + F[x]*Yeq[x])^2 +  Derivative[1][Yeq][x]/(1 + F[x]*Yeq[x])"
expr = expr.replace('Derivative[1][F][x]','dFdx[x]').replace('Derivative[1][Yeq][x]','dYeqdx[x]')
python_code = parse_mathematica(expr)
print(python_code)
python_simp = str(python_code).replace('F(x)*G(x)','fg').replace("/(F(x)*Yeq(x) + 1)","*t0").replace("Yeq(x)","yeq")
print(python_simp)

-F(x)*G(x)**2 - 2*F(x)*G(x)*Yeq(x) - dYeqdx(x) + 2*F(x)*G(x)*Yeq(x)/(F(x)*Yeq(x) + 1) + 2*F(x)*Yeq(x)**2/(F(x)*Yeq(x) + 1) + dYeqdx(x)/(F(x)*Yeq(x) + 1) - F(x)*Yeq(x)**2/(F(x)*Yeq(x) + 1)**2 - F(x)*Yeq(x)*dYeqdx(x)/(F(x)*Yeq(x) + 1)**2 - Yeq(x)**2*dFdx(x)/(F(x)*Yeq(x) + 1)**2
-fg**2 - 2*fg*yeq - dYeqdx(x) + 2*fg*yeq*t0 + 2*F(x)*yeq**2*t0 + dYeqdx(x)*t0 - F(x)*yeq**2*t0**2 - F(x)*yeq*dYeqdx(x)*t0**2 - yeq**2*dFdx(x)*t0**2


In [21]:
def Yeq(x):
    #m represents particle's mass, T is temperature in GeV, g represents the particle's degrees of freedom, 
    #and s represents entropy density.
    m = 1.0
    mDM = 1.0
    g = 1.0
    T = mDM/x

    g_star = 1.0 #g_* is the number of relativistic degrees of freedom
    s = 1/x**3 #entropy density
    if (m/T)>10: #non-relativistic regime
        Y_eq = (45/(4 * np.pi**4)) * (abs(g)/g_star) * (m/T)**(1.5) * np.sqrt(np.pi/2) * np.exp(-m/T) * (1 + (15/(8 * (m/T))) + 105/(128 * (m/T)**2) - (315/(1024 * (m/T)**3) )) #(abs(g) * (m * T/ (2 * np.pi))**(3/2) * np.exp(-m/T))/s
    elif (m/T)>(2./3.): #semi-relativistic regime
        Y_eq = ((abs(g)/(2 * np.pi**2)) * ( T * m**2)* kn(2, m/T)/s) #(45 * abs(g) * (m/T)**2 * kn(2,(m/T)))/(4 * np.pi**4 * g_star)
    elif (m/T)<=(2/3): #relativistic regime
        if g > 0: #fermions
           Y_eq = ((3/4) * (zeta(3)/np.pi**2) * abs(g) * (T)**3)/s
        if g < 0: #bosons
            Y_eq = ((zeta(3)/np.pi**2) * abs(g) * (T)**3)/s
    return Y_eq

def F(x,l=10.0):

    return l**2/x**2

def dYeqdx(x):

    dx = 1e-4
    return (Yeq(x+dx)-Yeq(x))/dx

def dFdx(x):

    return (-2/x**3)*F(1)

def YtoG(Y,x):
    f = F(x)
    t1 = f/(1+f)
    return Y - Yeq(x)*t1

def GtoY(G,x):
    f = F(x)
    return G + Yeq(x)*(1-1/(1+f))

def dYdx(Y,x):

    f = F(x)
    
    return f*(Yeq(x)**2 - Y**2)


def Y_app(xfout,l=10.0):
    Yf = xfout/l**2
    return Yf

In [22]:
def dGdx(G,x):

    f = F(x)
    yeq = Yeq(x)
    fg = f*G
    
    t0 = 1/(1+f)
    # t1 = f*t0
    # t12 = f*t0**2    
    # d = -fg*G -2*fg*yeq + 2*t1*G*yeq + (2*t1-t12)*yeq**2 - t0**2*yeq*dFdx(x) - dYeqdx(x)*(t0-1) 

    d = -fg**2 - 2*fg*yeq - dYeqdx(x) + 2*fg*yeq*t0 + 2*F(x)*yeq**2*t0 + dYeqdx(x)*t0 - F(x)*yeq**2*t0**2 - F(x)*yeq*dYeqdx(x)*t0**2 - yeq**2*dFdx(x)*t0**2
    return d

In [23]:
x0 = 0.5
Y0 = Yeq(x0)
G0 = YtoG(Y0,x0)
xvals = np.linspace(x0,20.,1000)

print(Yeq(x0),F(x0),dYeqdx(x0),dFdx(x0),G0,dGdx(G0,x0))

0.09134537117517981 400.0 0.0 -1600.0 0.0002277939430802428 -0.008198632719638643


In [24]:
G_sol,gDict = odeint(dGdx,G0,xvals,full_output=True)
Y_sol0,yDict = odeint(dYdx,Y0,xvals,full_output=True)

In [25]:
Y_sol = [GtoY(G,x) for G,x in zip(G_sol,xvals)]
Y_eq = [Yeq(x) for x in xvals]
dGdx_sol = [dGdx(G,x) for G,x in zip(G_sol,xvals)]
dYdx_sol = [dYdx(Y,x) for Y,x in zip(Y_sol0,xvals)]

In [26]:
plt.plot(xvals,Y_sol,label='Y from dGdx')
plt.plot(xvals,Y_sol0,label='Y from dYdx')
plt.plot(xvals,Y_eq,linestyle='dashed',label='Yeq')
plt.plot(xvals,dGdx_sol,linestyle='dotted',label='dGdx')
plt.plot(xvals,dYdx_sol,linestyle='dotted',label='dYdx')

plt.legend()
plt.yscale('log')
plt.xscale('log')
# plt.ylim(1e-10,1e-1)
plt.ylim(1e-4,10)
plt.grid()
plt.show()

In [27]:
print(gDict['nst'].max(),yDict['nst'].max())
print(gDict['nfe'].max(),yDict['nfe'].max())
print(len(gDict['nst']),len(yDict['nst']))

357 139
796 294
999 999


In [28]:
xvals[-1],Y_sol0[-1],Y_sol[-1],Yeq(xvals[-1])

(20.0, array([0.02061306]), array([0.01274806]), 2.9240493057623725e-08)

In [29]:
# plt.plot(xvals,Y_sol,label='Y from dGdx')
# plt.plot(xvals,Y_sol0,label='Y from dYdx')
# plt.plot(xvals,Y_eq,linestyle='dashed',label='Yeq')
plt.plot(xvals,dGdx_sol,linestyle='dotted',label='dGdx')
plt.plot(xvals,dYdx_sol,linestyle='dotted',label='dYdx')

plt.legend()
# plt.yscale('log')
plt.xscale('log')
# plt.ylim(1e-10,1e-1)
# plt.ylim(1e-4,10)
plt.grid()
plt.show()